### OpenApi Covid 크롤러

In [ ]:
datatest = "{0}{1:0>2}{2:0>2}".format(str(2023), str(12), str(31))

In [ ]:
type(datatest)

In [ ]:
for i in range(1, 13):
  print(i)

In [ ]:
respnseDatatest = getRequestUrl("https://apis.data.go.kr/1352000/ODMS_COVID_02/callCovid02Api?apiType=JSON&serviceKey=PrnCXubPpC7QR6yoJ3Fe2us5MIA%2Bcr8zHjEm0Er26tGnMHOFTv9lz47fAygM%2Fo4cTZZroZwtX9t2NjeEADvfgw%3D%3D&status_dt=20211028")

In [ ]:
getCovidStatsItem("20200932")


## 필요한 패키지 import

In [55]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

### ServiceKey 정보 입력

In [56]:
serviceKey = "PrnCXubPpC7QR6yoJ3Fe2us5MIA%2Bcr8zHjEm0Er26tGnMHOFTv9lz47fAygM%2Fo4cTZZroZwtX9t2NjeEADvfgw%3D%3D"

### 크롤러에 필요한 함수 선언

In [57]:
#[Code 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            print(response.headers)
            return response.read().decode("utf-8")
    except Exception as e:
            print(e)
            print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
            return None

In [58]:
#[CODE 2]
def getCovidStatsItem(status_dt):
    service_url = "http://apis.data.go.kr/1352000/ODMS_COVID_02/callCovid02Api"
    parameters = "?apiType=JSON&serviceKey=" + serviceKey
    parameters += "&status_dt=" + status_dt
    # parameters += "&numOfRows=" + numOfRows
    # parameters += "&pageNo=" + pageNo

    url = service_url + parameters

    response_data = getRequestUrl(url)
    
    
    if (response_data == None):
        return None
    else:
        return json.loads(response_data)


In [64]:
#[CODE 3]
def getCovidstatsService(statusDt, endStatusDt):
    jsonResult=[]
    result = []
    # statusDt = str(statusDt)
    # endStatusDt = str(endStatusDt)
    dataEnd = "{0}{1:0>2}{2:0>2}".format(str(endStatusDt), str(12), str(31))
    isDataEnd = 0

    for year in range(statusDt, endStatusDt+1):
        for month in range(10, 11):
          for date in range(28, 31):
            if(isDataEnd == 1) : break
            startDate = "{0}{1:0>2}{2:0>2}".format(str(year), str(month), str(date))
            jsonData = getCovidStatsItem(startDate)

            if(jsonData['resultMsg'] == 'NORMAL SERVICE'):
                if jsonData['items'] == '':
                    isDataEnd = 1
                    dataEnd = "{0}{1:0>2}{2:0>2}".format(str(year), str(month), str(date))
                    print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월 %s일 까지입니다." % (str(year), str(month), str(date)))
                    break
                    
                print(json.dumps(jsonData, indent=4, sort_keys=True, ensure_ascii=False))

                pageNo = jsonData['pageNo']
                resultCode = jsonData['resultCode']
                totalCount = jsonData['totalCount']
                
                accDefRate = jsonData['items'][0]['accDefRate']
                accExamCnt = jsonData['items'][0]['accExamCnt']
                accExamCompCnt = jsonData['items'][0]['accExamCompCnt']
                careCnt = jsonData['items'][0]['careCnt']
                dPntCnt = jsonData['items'][0]['dPntCnt']
                gPntCnt = jsonData['items'][0]['gPntCnt']
                hPntCnt = jsonData['items'][0]['hPntCnt']
                resutlNegCnt = jsonData['items'][0]['resutlNegCnt']
                statusDt = jsonData['items'][0]['statusDt']
                statusTime = jsonData['items'][0]['statusTime']
                uPntCnt = jsonData['items'][0]['uPntCnt']

                numOfRows = jsonData['numOfRows']
                resultMsg = jsonData['resultMsg']
                
                print('----------------------------------------------------------------------')
                jsonResult.append({'pageNo' : pageNo,
                                   'resultCode' : resultCode,
                                   'totalCount': totalCount,
                                   'items' : [{
                                       'accDefRate' : accDefRate,
                                       'accExamCnt' : accExamCnt,
                                       'accExamCompCnt' :accExamCompCnt,
                                        'careCnt' : careCnt,
                                        'dPntCnt' : dPntCnt,
                                        'gPntCnt' : gPntCnt,
                                        'hPntCnt': hPntCnt,
                                        'resultNegCnt' : resutlNegCnt,
                                        'statusDt' : statusDt,
                                        'statusTime' : statusTime ,
                                        'uPntCnt' : uPntCnt
                                   }],
                                    'numOfRows' : numOfRows,
                                    'resultMsg' : resultMsg
                                })
                result.append(jsonResult)
                    
    return (jsonResult, result, dataEnd, totalCount)

In [65]:
def main():
    jsonResult = []
    result = []
    print("보건복지부_코로나19 감염현황 총괄 통계 데이터를 수집합니다.")
    # pageNo = input('페이지 번호를 입력하세요')
    # numOfRows = input('한 페이지에 표시되는 데이터 수를 입력하세요')
    statusDt = int(input('데이터를 몇 년부터 수집할까요? : '))
    endStatusDt = int(input('데이터를 몇 년까지 수집할까요? : '))

    jsonResult, result, dataEnd, totalCount = getCovidstatsService(statusDt, endStatusDt)

    if (totalCount==0) : #URL 요청은 성공하였지만, 데이터 제공이 안된 경우
        print('데이터가 전달되지 않았습니다. 공공데이터포털의 서비스 상태를 확인하기 바랍니다.')
    else:
        with open('./%d_%d.json' % (statusDt, endStatusDt), 'w', encoding='utf8') as outfile:
            jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
            outfile.write(jsonFile)
        # colums = ['totalCount', 'hPntCnt', 'statusDt', 'endStatusDt']
        # result_df = pd.DataFrame(result, columns=colums)
        # result_df.to_csv('./%d_%d_%s_%s.csv' %(totalCount, statusDt, endStatusDt), index=False, encoding='cp949')

### 실행

In [66]:
if __name__ == '__main__':
    main()

보건복지부_코로나19 감염현황 총괄 통계 데이터를 수집합니다.
[2024-04-29 23:09:02.347606] Url Request Success
Set-Cookie: JSESSIONID=iTc3T0le_bPL9IIFeVuRO7HaVxVCZ3WF7hK78TES.ODMS4; Path=/1352000/ODMS_COVID_02; Domain=apis.data.go.kr
Access-Control-Allow-Origin: *
Content-Type: application/json;charset=UTF-8
Content-Length: 338
Date: Mon, 29 Apr 2024 14:09:02 GMT
Connection: close
Server: NIA API Server


{
    "items": [
        {
            "accDefRate": "1.021305784",
            "accExamCnt": "2582960",
            "accExamCompCnt": "2560056",
            "careCnt": "1617",
            "dPntCnt": "24068",
            "gPntCnt": "461",
            "hPntCnt": "26146",
            "resutlNegCnt": "2533910",
            "statusDt": "20201028",
            "statusTime": "00:00",
            "uPntCnt": "22904"
        }
    ],
    "numOfRows": "500",
    "pageNo": "1",
    "resultCode": "00",
    "resultMsg": "NORMAL SERVICE",
    "totalCount": 1
}
-----------------------------------------------------------------